In [32]:
import logging
import requests
from bs4 import BeautifulSoup
import re
import json
import urllib.parse
from concurrent.futures import ThreadPoolExecutor

In [2]:
import torch
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer
import nltk

In [46]:
def clean_duplicates(input_filename, output_filename):
    """Удаляет дубликаты из JSON файла и записи с '/Gallery' или ':Canon' в заголовке, затем сохраняет результат."""
    try:
        with open(input_filename, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        seen = set()
        unique_data = []
        for entry in data:
            # Проверка наличия '/Gallery' в заголовке
            title = entry.get("title", "").strip()
            if "/Gallery" in title:
                logging.info(f"Статья '{title}' удалена из-за наличия '/Gallery'.")
                continue
            # Проверка наличия ':Canon' в заголовке
            if ":Canon" in title:
                logging.info(f"Статья '{title}' удалена из-за наличия ':Canon'.")
                continue
            
            # Проверка уникальности по нормализованному заголовку
            key = title.lower()  # Нормализуем ключ
            if key not in seen:
                seen.add(key)
                unique_data.append(entry)
        
        with open(output_filename, "w", encoding="utf-8") as f:
            json.dump(unique_data, f, ensure_ascii=False, indent=4)
        logging.info(f"Очищенные данные сохранены в файл {output_filename}. Уникальных записей: {len(unique_data)}")
    except Exception as e:
        logging.error(f"Ошибка при обработке файла: {e}")

In [47]:
clean_duplicates("transformed_data.json", "cleaned_data.json")

In [48]:
def normalize_name(name):
    """
    Нормализует строку: удаляет лишние символы и приводит к CapitalCase.
    """
    if not isinstance(name, str):
        return name
    
    # Убираем лишние символы и заменяем разделители на пробелы
    name = (
        name.strip()
        .replace("&", "and")
        .replace("(", "")
        .replace(")", "")
        .replace("'", "")
        .replace(",", "")
        .replace(".", "")
        .replace("-", " ")
        .replace("_", " ")
        .replace("/", " ")
    )
    
    # Приводим каждое слово в строке к CapitalCase
    words = re.split(r'\s+', name)  # Разделяем по пробелам
    return "".join(word.capitalize() for word in words)

def consolidate_and_clean_categories(entity, mapping, irrelevant_keywords_per_class, major_authors):
    """
    Схлопывает категории, удаляет нерелевантные и форматирует категории в CapitalCase.
    """
    if "subclass" in entity and isinstance(entity["subclass"], list):
        updated_subclasses = set()
        class_name = normalize_name(entity.get("class", ""))  # Нормализуем класс

        for subclass in entity["subclass"]:
            normalized_subclass = normalize_name(subclass)  # Нормализуем подкатегорию
            if normalized_subclass == entity["title"]:  # Проверяем на самоссылку
                continue

            # Применяем CATEGORY_MAPPING без повторной нормализации результата
            matched = False
            for key, replacement in mapping.items():
                if key.lower() in normalized_subclass.lower():
                    updated_subclasses.add(replacement)  # Берём замену напрямую из mapping
                    matched = True
                    break
            if not matched:
                updated_subclasses.add(normalized_subclass)

        # Удаление нерелевантных категорий
        if class_name in irrelevant_keywords_per_class:
            updated_subclasses = {
                sub for sub in updated_subclasses
                if not any(irrelevant.lower() in sub.lower() for irrelevant in irrelevant_keywords_per_class[class_name])
            }

        # Обработка авторов
        if class_name == "Work":
            non_major_authors = {
                sub for sub in updated_subclasses if sub.endswith("Works") and sub not in major_authors
            }
            if non_major_authors:
                updated_subclasses -= non_major_authors
                updated_subclasses.add("OtherAuthors")

        # Сохраняем результат
        entity["subclass"] = sorted(updated_subclasses)  # Упорядочиваем для консистентности
    return entity


# Карта замены для схлопывания подкатегорий
CATEGORY_MAPPING = {
    "SpeciesOriginatingFrom": "Species",
    "CharactersOriginatingFrom": "MythosCharacters",
    "AvatarsOf": "Avatars",
    "Games": "GamesAndAdaptations",
    "Novels": "MythosLiterature",
    "Anthologies": "MythosLiterature",
    "LovecraftCircleWorks": "MythosLiterature",
    "ExpandedMythosWorks": "MythosLiterature",
    "ExpandedMythosFilmAndTelevisionAdaptations": "MediaAdaptations",
    "MythosInspiredFilmAndTelevisionWorks": "MediaAdaptations"
}

IRRELEVANT_KEYWORDS_PER_CLASS = {
    "Character": ["Artefacts", "Locations", "Planets", "RealWorld", "Upcoming", "CthulhuArmageddon"],
    "Work": [
        "Artefacts", "Locations", "Planets", "RealWorld", "Upcoming", "CthulhuArmageddon",
        "TheDandridgeCycle", "TheArkhamDetective", "TheBookOfChaos"
    ],
    "Location": ["Averoigne"]
}

MAJOR_AUTHORS = [
    "HPLovecraftWorks", "AugustDerlethWorks", "ClarkAshtonSmithWorks", 
    "RobertEHowardWorks", "RamseyCampbellWorks", "BrianLumleyWorks", 
    "FritzLeiberWorks", "CaitlinRKiernanWorks"
]


# Загрузка данных
with open("cleaned_data.json", "r", encoding="utf-8") as file:
    cleaned_data = json.load(file)

# Применение обновлений к данным
for i, entity in enumerate(cleaned_data):
    if entity.get("class") in ["Character", "Work", "RealWorldPerson", "Location", "Artefact", "Organisation"]:
        cleaned_data[i] = consolidate_and_clean_categories(
            entity, CATEGORY_MAPPING, IRRELEVANT_KEYWORDS_PER_CLASS, MAJOR_AUTHORS
        )

# Сохранение обновлённых данных
output_file = "cleaned_data_updated.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print(f"Updated data saved to '{output_file}'")

Updated data saved to 'cleaned_data_updated.json'


In [ ]:
data_path = 'cleaned_data_updated.json'
filtered_data_path = 'filtered_data.json'

with open(data_path, 'r') as data_file:
    cleaned_data = json.load(data_file)

# Фильтрация данных: исключение Unclassified
filtered_data = [entity for entity in cleaned_data if entity.get("class") != "Unclassified"]

with open(filtered_data_path, 'w') as filtered_file:
    json.dump(filtered_data, filtered_file, indent=4, ensure_ascii=False)

print(f"Entities with class 'Unclassified' have been removed. Filtered data saved to '{filtered_data_path}'.")

In [ ]:
nltk.download('punkt')

class TextSummarizer:
    def __init__(self, model_name="t5-large", device="cuda:0"):
        """
        Инициализация модели и токенизатора.
        :param model_name: Название предобученной модели.
        :param device: Устройство для выполнения ('cuda:0' для GPU).
        """
        self.device = torch.device(device)
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def summarize(self, text, reduction_ratio=0.7):
        """
        Генерация саммари с учётом заданного процента сокращения.
        :param text: Исходный текст для саммаризации.
        :param reduction_ratio: Процент оставляемого текста (от 0 до 1).
        :return: Сгенерированное саммари.
        """
        # Нормализация текста
        text = self.clean_empty_quotes(text)

        num_words = len(text.split())
        target_length = max(1, int(num_words * reduction_ratio))
        
        input_text = "summarize: " + text
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(self.device)

        summary_ids = self.model.generate(
            input_ids, 
            max_length=target_length, 
            length_penalty=2.0, 
            num_beams=4, 
            early_stopping=True
        )
        
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return self.fix_sentence_case(summary)

    @staticmethod
    def clean_empty_quotes(text):
        """
        Удаляем пустые кавычки и сочетания вида " " из текста.
        """
        text = re.sub(r'\s*""\s*', '', text)
        text = re.sub(r'\s*"\s*"\s*', '', text)
        return text

    @staticmethod
    def fix_sentence_case(text):
        """
        Исправляем регистр первой буквы каждого предложения.
        """
        sentences = nltk.sent_tokenize(text)
        fixed_sentences = [sentence.capitalize() for sentence in sentences]
        return " ".join(fixed_sentences)

def count_sentences(text):
    """Считаем количество предложений в тексте."""
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

def summarize_json_content(input_filename, output_filename, summary_log_filename, summarizer):
    """
    Применяем суммаризатор к полю content в JSON-файле и сохраняем результат в отдельный лог-файл.
    :param input_filename: Имя входного файла JSON.
    :param output_filename: Имя выходного файла JSON.
    :param summary_log_filename: Имя файла для записи саммари с отбивкой.
    :param summarizer: Экземпляр класса TextSummarizer.
    """
    try:
        with open(input_filename, "r", encoding="utf-8") as f:
            data = json.load(f)

        with open(summary_log_filename, "w", encoding="utf-8") as log_file:
            for entry in tqdm(data, desc="Обработка записей", unit="запись"):
                if "content" in entry:
                    original_content = entry["content"]
                    num_sentences = count_sentences(original_content)

                    # Если предложений 5 или меньше, не сокращаем
                    if num_sentences <= 5:
                        log_file.write(f"+++++++\n{original_content}\n+++++++\n\n")
                        continue

                    try:
                        # Сокращаем текст на 30%
                        summarized_content = summarizer.summarize(original_content, reduction_ratio=0.7)
                        entry["content"] = summarized_content
                        # Записываем саммари в лог-файл с отбивкой
                        log_file.write(f"+++++++\n{summarized_content}\n+++++++\n\n")
                    except Exception as e:
                        logging.error(f"Ошибка суммаризации для записи '{entry.get('title', 'без названия')}': {e}")
        
        with open(output_filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        logging.info(f"Сокращённые данные сохранены в файл {output_filename}.")
    except Exception as e:
        logging.error(f"Ошибка обработки файла: {e}")

logging.basicConfig(level=logging.INFO)
summarizer = TextSummarizer(model_name="t5-large", device="cuda:0")
summarize_json_content("filtered_data.json", "summarized_data.json", "summaries_log.txt", summarizer)